<a href="https://colab.research.google.com/github/Rekt77/kisa_insuretech/blob/master/pytorch_resnet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RESNET34를 이용한 자동차 사진 분류

![why resnet](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FdrYEZ4%2FbtqwpSXYd8U%2FLF8RkIhWpAmsgR67nfnZsk%2Fimg.png)

- 이미지 분류 대회에서 2015년도에 resnet이 좋은 결과를 보임
- 레이어가 깊어 질수록 좋은 효과를 보이는 경향이 있음

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

import time
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name(device))

cuda:0
Tesla P100-PCIE-16GB


In [2]:
# 자동차 데이터 저장경로
dataset_dir = "/content/drive/My Drive/insuretech/car/origin_data/"

# 데이터 전처리
# randomhorizontalFlip() : 이미지 좌우반전하여 학습
# randomRotation() : 이미지 회전하여 학습
# Normalize() : 이미지를 0과 0.5 사이 값으로 정규화 한후 (pixel - 0.5) / 0.5 진행 -> 결과 == -1과 1사이의 값
train_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(15),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = torchvision.datasets.ImageFolder(root=dataset_dir+"train_data", transform = train_tfms)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 10, shuffle=True, num_workers = 2)

dataset2 = torchvision.datasets.ImageFolder(root=dataset_dir+"test_data", transform = test_tfms)
testloader = torch.utils.data.DataLoader(dataset2, batch_size = 10, shuffle=False, num_workers = 2)

In [3]:
def train_model(model, criterion, optimizer, scheduler, n_epochs = 5):
    
    losses = []
    accuracies = []
    test_accuracies = []

    # set the model to train mode
    model.train()

    # train
    for epoch in range(n_epochs):

      # duration 체크
        since = time.time()
        running_loss = 0.0
        running_correct = 0.0
        for i, data in enumerate(trainloader, 0):
            # 배치사이즈 크기 만큼의 사진을 받아오고 데이터를 inputs, 라벨을 labels에 저장
            inputs, labels = data

            #cuda에 변수 할당
            inputs = inputs.to(device)
            labels = labels.to(device)

            #기울기 초기화
            optimizer.zero_grad()
            
            # hypothesis
            outputs = model(inputs)

            # 출력은 11개 분류 각각에 대한 값으로 나타납니다.
            # 어떤 분류에 대해서 더 높은 값이 나타난다는 것은, 신경망이 그 이미지가 해당 분류에 더 가깝다고 생각한다는 것입니다
            _, predicted = torch.max(outputs.data, 1)

            # loss값 구하기
            loss = criterion(outputs, labels)

            # loss 최소화
            loss.backward()
            optimizer.step()
            
            # calculate the loss/acc later
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()


        # loss 값 구하기
        epoch_duration = time.time()-since
        epoch_loss = running_loss/len(trainloader)

        # running_correct/trainloader = 전체 데이터 중에 몇개를 맞추었는지
        # trainloader는 배치사이즈 10으로 구성되어 있어서 실제 데이터보다 10분의 1크기
        epoch_acc = 100.0*(running_correct/(len(trainloader)*10))
        print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
        
        #losses.append(epoch_loss)
        #accuracies.append(epoch_acc)
        
        # switch the model to eval mode to evaluate on test data
        model.eval()
        test_acc = eval_model(model)
        #test_accuracies.append(test_acc)
        
        # re-set the model to train mode after validating
        model.train()
        scheduler.step(test_acc)
        since = time.time()
    print('Finished Training')
    return model#, losses, accuracies, test_accuracies

In [4]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * correct / total
    print('Accuracy of the network on the test images: %d %%' % (
        test_acc))
    return test_acc

In [5]:
# resnet을 불러옴
model_ft = models.resnet34(pretrained=True)

# resnet convolution layer를 통과한 데이터가 몇개의 피쳐를 가지고 있는지
# 선형결합을 위해 필요한 부분
num_ftrs = model_ft.fc.in_features

# 11개의 클래스로 분류하기 위함
model_ft.fc = nn.Linear(num_ftrs, 11)
model_ft = model_ft.to(device)

# cross entropy loss를 구함
criterion = nn.CrossEntropyLoss()

# 경사하강법으로 최적화
optimizer = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)

# learning rate를 높이거나 낮추는 스케줄러
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

In [6]:
# 학습 루틴 시작
model_ft = train_model(model_ft, criterion, optimizer, lrscheduler, n_epochs=10)

Epoch 1, duration: 7 s, loss: 2.6155, acc: 23.4091
Accuracy of the network on the test images: 10 %
Epoch 2, duration: 7 s, loss: 2.2497, acc: 35.9091
Accuracy of the network on the test images: 36 %
Epoch 3, duration: 7 s, loss: 1.4845, acc: 55.0000
Accuracy of the network on the test images: 39 %
Epoch 4, duration: 7 s, loss: 1.4396, acc: 57.0455
Accuracy of the network on the test images: 50 %
Epoch 5, duration: 7 s, loss: 0.8645, acc: 70.9091
Accuracy of the network on the test images: 54 %
Epoch 6, duration: 7 s, loss: 0.7341, acc: 76.3636
Accuracy of the network on the test images: 48 %
Epoch 7, duration: 7 s, loss: 0.5073, acc: 82.2727
Accuracy of the network on the test images: 82 %
Epoch 8, duration: 7 s, loss: 0.2279, acc: 92.5000
Accuracy of the network on the test images: 85 %
Epoch 9, duration: 7 s, loss: 0.1513, acc: 95.6818
Accuracy of the network on the test images: 86 %
Epoch 10, duration: 7 s, loss: 0.1521, acc: 94.7727
Accuracy of the network on the test images: 87 %

In [8]:
# tie the class indices to their names

def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx
classes, c_to_idx = find_classes(dataset_dir+"train_data")

In [9]:
c_to_idx

{'Hyundai Accent Sedan 2012': 0,
 'Hyundai Azera Sedan 2012': 1,
 'Hyundai Elantra Sedan 2007': 2,
 'Hyundai Elantra Touring Hatchback 2012': 3,
 'Hyundai Genesis Sedan 2012': 4,
 'Hyundai Santa Fe SUV 2012': 5,
 'Hyundai Sonata Hybrid Sedan 2012': 6,
 'Hyundai Sonata Sedan 2012': 7,
 'Hyundai Tucson SUV 2012': 8,
 'Hyundai Veloster Hatchback 2012': 9,
 'Hyundai Veracruz SUV 2012': 10}

In [7]:
total = 0
correct = 0

model_ft.eval()
for imgs,labels in testloader:
  imgs = imgs.to(device)
  labels = labels.to(device)
  hypothesis = model_ft(imgs).to(device)
  predicted = torch.argmax(hypothesis.data, 1)
  print(predicted)
  print(labels)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()

print('Accuracy:', 100 * correct / total)

tensor([0, 0, 0, 0, 3, 0, 9, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 8, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([ 0, 10,  0,  0,  1,  1,  1,  1,  1,  1], device='cuda:0')
tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 7, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([1, 1, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([0, 2, 2, 0, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 0, 2, 0, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0')
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3],